In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import re
from tqdm import trange, tqdm

импорт библиотек

In [2]:
reqx = requests.get('https://hi-tech.md/kompyuternaya-tehnika/sport-i-otdyh/hd-cvi-kamery/ip-kamery/')

In [3]:
soup = bs(reqx.text, 'html.parser')

In [37]:
data = pd.DataFrame({'soup_name':pd.Series(soup.findAll('div', class_='ypi-grid-list__item_body'),dtype = 'object')})
data.head()

,soup_name
0,"[[ , [\n, <input name=""appearance[show_sku]"" t..."
1,"[[ , [\n, <input name=""appearance[show_sku]"" t..."
2,"[[ , [\n, <input name=""appearance[show_sku]"" t..."
3,"[[ , [\n, <input name=""appearance[show_sku]"" t..."
4,"[[ , [\n, <input name=""appearance[show_sku]"" t..."


парсинг тестовой страници сайта 

In [38]:
t = pd.DataFrame({'soup_name':pd.Series(soup.findAll('div', class_='ypi-grid-list__item_body'))}).loc[1,'soup_name']
t

<div class="ypi-grid-list__item_body" style="min-height: 430px"><div class="stock-block"> <div class="ty-control-group ty-sku-item cm-hidden-wrapper cm-reload-42964" id="sku_update_42964">
<input name="appearance[show_sku]" type="hidden" value="1"/>
<label class="ty-control-group__label" id="sku_42964">КОД:</label>
<span class="ty-control-group__item">Т-000072433</span>
</div>
<div class="cm-reload-42964 stock-wrap" id="product_amount_update_42964">
<input name="appearance[show_product_amount]" type="hidden" value="1"/>
<a class="ty-control-group product-list-field cm-external-click" data-ca-external-click-id="availability_in_stores" href="#availability_in_stores">
<span class="ty-qty-in-stock preorder ty-control-group__item" id="in_stock_info_42964">
<i class="material-icons"></i>Предзаказ
</span>
</a>
</div>
</div><div class="ty-grid-list__image" style="height: 230px;">
<div class="ypi-list-bt-block hidden">
<a class="ty-btn ty-btn__tertiary ty-add-to-wish cm-submit text-button" dat

создание временной переменной для поиска полей для парсинга

In [7]:
def names_correct(x):
    name = str(x)
    return  name 

In [8]:
data['soup_name'] = data['soup_name'].apply(names_correct)

меняем данные парсинга на текстовый тип 

In [9]:
def find_id(x):
    find = x.find('span class="ty-control-group__item"')
    start = find+len('span class="ty-control-group__item"')+1
    stop = find+12+ len('span class="ty-control-group__item"')
    xxx = x[start: stop]
    return xxx

In [10]:
data['id'] = data['soup_name'].apply(find_id)

находим в тексте и созаем столбец с кодами товара

In [11]:
data['date'] = pd.to_datetime('today').normalize()

создаем столбец с датой парсинга

In [12]:
def find_availability(t):
    find = t.find('i class="material-icons"')
    find = t.find('</i>', find)
    start = find+len('</i>')
    stop = t.find('\n', find)
    xxx = t[start: stop]
    return xxx

In [13]:
data['availability'] = data['soup_name'].apply(find_availability)

находим в тексте и создаем столбец с наличием товара

In [14]:
def find_title(t):
    find = t.find('a class="product-title"')
    find = t.find('title=', find)
    start = find+len('title=')+1
    stop = t.find('>', find)-1
    xxx = t[start: stop]
    return xxx


In [15]:
data['title'] = data['soup_name'].apply(find_title)

находим в тексте и создаем столбец снаименованием товара

In [16]:
def find_price(t):
    find = t.find('span class="ty-price-num"')
    find = t.find('>', find)
    start = find+len('>')
    stop = t.find('<', find)
    xxx = t[start: stop]
    xxx = xxx.replace(u'\xa0',u'')
    return xxx

In [17]:
data['price'] = data['soup_name'].apply(find_price)

Находим в тексте и создаем столбец с ценой товара

In [18]:
def find_old_price(t):
    find = t.find('span class="ty-strike"')
    find = t.find('span class="ty-list-price ty-nowrap"',find)
    find = t.find('>', find)
    start = find+len('>')
    stop = t.find('<', find)
    xxx = t[start: stop]
    xxx = xxx.replace(u'\xa0',u'')
    return xxx

In [19]:
data['old_price'] = data['soup_name'].apply(find_old_price)

Для акционного товара Находим в тексте и создаем столбец со старой ценой до начала акции

In [20]:
def find_url(t):
    find = t.find('a class="product-title"')
    find = t.find('href="', find)
    start = find+len('href="')
    stop = t.find('title', find)-2
    xxx = t[start: stop]
    return xxx

In [21]:
data['soup_name'] = data['soup_name'].apply(find_url)

Находим в тексте информацию об адресе совара и переписываем эту информацию вместо парсинговой.

In [22]:
def preprocessing(x):
    print(f'таблица имеет следующий вид:')
    display(x.head())
    print('*'*100)
    print(f'таблица имеет {x.shape[0]} сторок и {x.shape[1]} столбцов')
    print('*'*100)
    print(f'в таблице обнаружено дублекатов:{x.duplicated().sum()}')
    print('*'*100)
    print('в таблице обнаружены пропуски в следующих столбцах:')
    display(x.isna().sum())
    print('*'*100)
    print('Эти пропуски составлябт следующее количество в процентах')
    display((x.isna().mean()* 100).round(1))
    print('*'*100)
    print('столбцы имеют следующие типы:')
    display(x.dtypes)
    x.columns = [y.lower().replace(' ', '_') for y in x.columns.values] 
    print('*'*100)
    print('названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание')
    print('*'*100)
    display(x.info())

создаем функцию предварительного просмотра данных

In [23]:
def parsing_2(url):
    data = pd.DataFrame(columns=['soup_name', 'id', 'date', 'availability', 'title', 'price', 'old_price'])
    for i in tqdm(url):        
        q = 1
        while q>0:        
            reqx = requests.get(i + str(q))
            soup = bs(reqx.text, 'html.parser')
            datax = pd.DataFrame({'soup_name':pd.Series(soup.findAll('div', class_='ypi-grid-list__item_body'),dtype = 'object')})
            data = pd.concat([data, datax], ignore_index=True)
            if len(list(soup.findAll('div', class_='ypi-grid-list__item_body'))) > 0:
                q = q + 1
            else:
                q = 0
    data['soup_name'] = data['soup_name'].apply(names_correct)
    data['id'] = data['soup_name'].apply(find_id)
    data['date'] = pd.to_datetime('today').normalize()
    data['availability'] = data['soup_name'].apply(find_availability)
    data['title'] = data['soup_name'].apply(find_title)
    data['price'] = data['soup_name'].apply(find_price)
    data['old_price'] = data['soup_name'].apply(find_old_price)
    data['soup_name'] = data['soup_name'].apply(find_url)
    data['price'] = data['price'].astype(float)
    data.loc[data['old_price'] == '','old_price'] = np.nan
    data['old_price'] = data['old_price'].astype(float)
    data = data.drop_duplicates()
    return data

создаем функцию, которая по списку url проходит и парсит всю доступную информацию, преобразуя ее в датафрейм.

In [24]:
url = ['https://hi-tech.md/televizory-i-elektronika/page-',
       'https://hi-tech.md/bytovaya-tehnika/page-',
       'https://hi-tech.md/kompyuternaya-tehnika/page-']

url = ['https://hi-tech.md/televizory-i-elektronika/page-']

список url по которым будет парсится сайт

In [25]:
df = parsing_2(url)
df

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [02:27<00:00, 49.09s/it]


,soup_name,id,date,availability,title,price,old_price
0,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088228,2024-05-10,В наличии,Батарейка Energenie EG-BA-CR1220-01,4.0,NaN
1,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088229,2024-05-10,В наличии,Батарейка Energenie EG-BA-CR2016-01,4.0,NaN
2,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088221,2024-05-10,В наличии,Батарейка Energenie EG-BA-AA4-01,5.0,NaN
3,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088222,2024-05-10,В наличии,Батарейка Energenie EG-BA-AAA4-01,5.0,NaN
4,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088230,2024-05-10,В наличии,Батарейка Energenie EG-BA-CR2025-01,5.0,NaN
...,...,...,...,...,...,...,...
6420,https://hi-tech.md/kompyuternaya-tehnika/tovar...,Т-000089380,2024-05-10,Предзаказ,"Моноблок Apple iMac 24"" 2023 Blue (MQRR3RU/A)",34799.0,38799.0
6424,https://hi-tech.md/kompyuternaya-tehnika/tovar...,Т-000089357,2024-05-10,Предзаказ,"Ноутбук Apple MacBook PRO 14.2"" 2023 Silver (M...",35399.0,NaN
6425,https://hi-tech.md/kompyuternaya-tehnika/tovar...,Т-000089359,2024-05-10,Предзаказ,"Ноутбук Apple MacBook PRO 14.2"" 2023 Silver (M...",39399.0,NaN
6426,https://hi-tech.md/kompyuternaya-tehnika/tovar...,Т-000089360,2024-05-10,Предзаказ,"Ноутбук Apple MacBook PRO 14.2"" 2023 Space Gre...",39399.0,NaN


полученная информация

data

In [26]:
preprocessing(df)

таблица имеет следующий вид:


,soup_name,id,date,availability,title,price,old_price
0,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088228,2024-05-10,В наличии,Батарейка Energenie EG-BA-CR1220-01,4.0,NaN
1,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088229,2024-05-10,В наличии,Батарейка Energenie EG-BA-CR2016-01,4.0,NaN
2,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088221,2024-05-10,В наличии,Батарейка Energenie EG-BA-AA4-01,5.0,NaN
3,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088222,2024-05-10,В наличии,Батарейка Energenie EG-BA-AAA4-01,5.0,NaN
4,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088230,2024-05-10,В наличии,Батарейка Energenie EG-BA-CR2025-01,5.0,NaN


****************************************************************************************************
таблица имеет 5884 сторок и 7 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


soup_name          0
id                 0
date               0
availability       0
title              0
price              0
old_price       4391
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


soup_name        0.0
id               0.0
date             0.0
availability     0.0
title            0.0
price            0.0
old_price       74.6
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


soup_name               object
id                      object
date            datetime64[ns]
availability            object
title                   object
price                  float64
old_price              float64
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5884 entries, 0 to 6427
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   soup_name     5884 non-null   object        
 1   id            5884 non-null   object        
 2   date          5884 non-null   datetime64[ns]
 3   availability  5884 non-null   object        
 4   title         5884 non-null   object        
 5   price         5884 non-null   float64       
 6   old_price     1493 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 367.8+ KB


None

In [27]:
today = pd.to_datetime('today').normalize()

In [28]:
today = today.to_pydatetime()

создаем переменную, которая хранит информацию о текущей дате

In [29]:
df.to_csv('csv/hi_tech_'+str(today.day)+'_'+str(today.month)+'_'+str(today.year)+'.csv', index=False)  

Переписывем датафрейм в файл

# пробные функции

In [30]:
t = data.loc[0,'soup_name']
t = str(t)
find = t.find('a class="product-title"')
find = t.find('href="', find)
start = find+len('href="')
stop = t.find('title', find)-2
xxx = t[start: stop]
xxx   

'://hi-tech.md/kompyuternaya-tehnika/sport-i-otdyh/hd-cvi-kamery/ip-kamery/hd-cvi-videokamera-dahua-dh-hac-hdw1230mp-028'

In [31]:
urls = df['soup_name'].unique()

In [32]:
reqx = requests.get('https://hi-tech.md/kompyuternaya-tehnika/sport-i-otdyh/hd-cvi-kamery/ip-kamery/')

In [33]:
soup = bs(reqx.text, 'html.parser')

In [34]:
data = pd.DataFrame({'soup_name':pd.Series(soup.findAll('div', class_='ypi-grid-list__item_body'),dtype = 'object')})
data.head()

,soup_name
0,"[[ , [\n, <input name=""appearance[show_sku]"" t..."
1,"[[ , [\n, <input name=""appearance[show_sku]"" t..."
2,"[[ , [\n, <input name=""appearance[show_sku]"" t..."
3,"[[ , [\n, <input name=""appearance[show_sku]"" t..."
4,"[[ , [\n, <input name=""appearance[show_sku]"" t..."


In [35]:
data.loc[0,'soup_name']

<div class="ypi-grid-list__item_body" style="min-height: 430px"><div class="stock-block"> <div class="ty-control-group ty-sku-item cm-hidden-wrapper cm-reload-42965" id="sku_update_42965">
<input name="appearance[show_sku]" type="hidden" value="1">
<label class="ty-control-group__label" id="sku_42965">КОД:</label>
<span class="ty-control-group__item">Т-000072435</span>
</input></div>
<div class="cm-reload-42965 stock-wrap" id="product_amount_update_42965">
<input name="appearance[show_product_amount]" type="hidden" value="1">
<a class="ty-control-group product-list-field cm-external-click" data-ca-external-click-id="availability_in_stores" href="#availability_in_stores">
<span class="ty-qty-in-stock preorder ty-control-group__item" id="in_stock_info_42965">
<i class="material-icons"></i>Предзаказ
</span>
</a>
</input></div>
</div><div class="ty-grid-list__image" style="height: 230px;">
<div class="ypi-list-bt-block hidden">
<a class="ty-btn ty-btn__tertiary ty-add-to-wish cm-submit te

In [36]:
t = pd.DataFrame({'soup_name':pd.Series(soup.findAll('div', class_="ty-warehouses-store__group"))}).loc[0,'soup_name']
t

C:\Users\user\AppData\Local\Temp\ipykernel_468\710482726.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  t = pd.DataFrame({'soup_name':pd.Series(soup.findAll('div', class_="ty-warehouses-store__group"))}).loc[0,'soup_name']


KeyError: 0

In [ ]:
list(soup.findAll('div', class_="ty-wysiwyg-content content-availability_in_stores"))

In [ ]:
soup